In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/FieldNN


#  PreCode

## DataLoader & Batch

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P-InfoGrn',  'P-EC-PNSect-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P-InfoGrn_wgt torch.Size([4, 43])
B-P-InfoGrn_tknidx torch.Size([4, 43])
B-P-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

## EmbedBlock

In [4]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'ReduceMax',
    'merger': 'MergeConcat',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################


######################
default_E_subunitName = 'EL'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
# learner_default_dict = {} # To update it in the future. 
######################

In [5]:
full_recfldgrn_list

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [6]:

df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunitName)
df_SubUnit['SubUnit_BasicNN_List'] = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName),  axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)
df_SubUnit['SubUnit_BasicNN_Config_List'] = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, ), axis = 1)
df_Embed_SubUnit = df_SubUnit
df_Embed_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,EL,[B-P-InfoGrn],B-P-Info,3,"[expander-CateEmbed, learner-TFM]","[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,EL,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,"[expander-LLMEmbed, learner-TFM]","[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '...","[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [7]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-Info torch.Size([4, 43, 128])
B-P-EC-PNSect-Tknz torch.Size([4, 23, 14, 221, 128])


## ReprBlock

In [8]:
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import generate_BasicNN_Config
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

#################
default_R_subunit_name = 'RL'
default_MR_subunit_name = 'ML'
#################

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
}
#############################################

#############################################
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################


In [9]:
full_recfldgrn_list_new = [i.replace('Grn', '') for i in full_recfldgrn_list]
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list_new)
df_dataflow_new = update_df_Repr_dataflow(df_dataflow)
df_dataflow = df_dataflow_new.copy()
df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit['SubUnit_BasicNN_List'] = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName),  axis = 1)
df_SubUnit['SubUnit_DefaultBasicNN_List'] = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)
df_SubUnit['SubUnit_BasicNN_Config_List'] = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

In [10]:
from fieldnn.module.reprblock import ReprBlockLayer

df_Repr_SubUnit = df_SubUnit
ReprBlock = ReprBlockLayer(df_SubUnit)

ValueError: There is no layer for "Max"

In [ ]:
# fld_updates_dict = {}
# for i in RECFLD_TO_EMBEDTESNOR:
#     layernum = len(i.split('-'))
#     fld = i.split('-')[-1]
#     if '@' in fld:
#         # print(fld)
#         neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
#         # print(neat_i)
#         same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
#         # print(same_neat_list)
#         if len(same_neat_list) == 1: # itself
#             # RECFLD_TO_EMBEDTESNOR[]
#             fld_updates_dict[i] = neat_i
            
# for old, new in fld_updates_dict.items():
#     RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
    
    
[i for i in RECFLD_TO_EMBEDTESNOR]

In [ ]:
OUTPUT_TO_TENSOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)
[i for i in OUTPUT_TO_TENSOR]

# RecModelPL

## Module

In [ ]:
import torch
import pytorch_lightning as pl
# from .embedblock import EmbedBlockLayer
# from .reprblock import ReprBlockLayer
from fieldnn.module.embedblock import EmbedBlockLayer
from fieldnn.module.reprblock import ReprBlockLayer


class RecModelPL(pl.LightningModule):
    """ Naive softmax-layer """
    def __init__(self, df_Embed_SubUnit, df_Repr_SubUnit, 
                 actn_fn_name, loss_fn_name,
                 output_name, embed_size, output_size):
        
        '''
            df_Embed_SubUnit: df_SubUnit for the Embed Block.
            df_Repr_SubUnit: df_Repr_SubUnit for the Repr Block.
            actn_fn_name: like sigmoid, softmax
            loss_fn_name: like CrossEntropy, BCELoss.
            output_name: the featvec name.
            embed_size: the featvec dim.
            output_size: the output dim size. if actn_fn is softmax, then label sets. if actn_fn is sigmoid, then 1. 
            
        '''
        super(RecModelPL, self).__init__()
        
        self.output_name = output_name
        self.EmbedBlock = EmbedBlockLayer(df_Embed_SubUnit)
        self.ReprBlock  = ReprBlockLayer(df_Repr_SubUnit)
        self.OutputBlock = torch.nn.Linear(embed_size, output_size)
        
        self.actn_fn_name = actn_fn_name
        if self.actn_fn_name == 'Sigmoid':
            self.actn_method = torch.nn.Sigmoid()
            self.actn_fn = lambda outputvecs: self.actn_method(outputvecs) # will return probs 
        elif self.actn_fn_name == 'Softmax':
            self.actn_method = torch.nn.Softmax()
            self.actn_fn = lambda outputvecs: self.actn_method(outputvecs, dim = 1) # will return probs 
        else:
            raise ValueError(f'Activation Function Name {actn_fn_name} is not available yet')
        
        self.loss_fn_name = loss_fn_name
        if self.loss_fn_name == 'BCELoss':
            assert self.actn_fn_name == 'Sigmoid'
            self.loss_method = torch.nn.BCELoss()
            self.loss_fn = lambda probs, targets: self.loss_method(probs, targets) # will return loss
        
        elif self.loss_fn_name == 'CrossEntropyLoss':
            assert self.actn_fn_name == 'Softmax'
            self.loss_method = torch.nn.CrossEntropyLoss()
            self.loss_fn = lambda probs, targets: self.loss_method(probs, targets) # will return loss
        
        else:
            raise ValueError(f'Loss Function Name {loss_fn_name} is not available yet')
        

    def get_REPR_TENSOR(self, batch_rfg):
        # (1) Embed Part
        # get the full_recfldgrn_list
        full_recfldgrn_list = list(set([i.split('_')[0] for i in batch_rfg]))

        # prepare RECFLD_TO_TENSOR
        RECFLD_TO_TENSOR = {}
        for full_recfldgrn in full_recfldgrn_list:
            RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
        # get RECLD_TO_EMBEDTENSOR from RECFLD_TO_TENSOR and EmbedBlock
        RECFLD_TO_EMBEDTESNOR = self.EmbedBlock(RECFLD_TO_TENSOR)
        
        # (2) Update RECFLD_TO_EMBEDTESNOR.
        # update the names of full_recfldgrn_list
            
        # (3) REPR Part
        # get the OUTPUT_TO_TENSOR data holder
        REPR_TENSOR = self.ReprBlock(RECFLD_TO_EMBEDTESNOR)
        return REPR_TENSOR

    def loss_funciton(self, batch_rfg, batch_targets):
        # batch -> Embed -> Repr -> repr_tensor
        REPR_TENSOR = self.get_REPR_TENSOR(batch_rfg)
        info_dict = REPR_TENSOR[self.output_name]
        featvecs = info_dict['info']
        outputvecs = self.OutputBlock(featvecs)
        probs = self.actn_fn(outputvecs) 
        batch_targets = batch_targets.float()
        l = self.loss_fn(probs, batch_targets)
        return l
        
    def forward(self, batch_rfg):
        # forward is the method for the inference.
        # batch -> Embed -> Repr -> repr tensor
        REPR_TENSOR = self.get_REPR_TENSOR(batch_rfg)
        info_dict = REPR_TENSOR[self.output_name]
        featvecs = info_dict['info']
        outputvecs = self.OutputBlock(featvecs)
        probs = self.actn_fn(outputvecs) 
        return probs
    
    ############# pl part.
    def training_step(self, batch, batch_idx):
        batch_rfg, batch_targets = batch
        loss = self.loss_funciton(batch_rfg, batch_targets)
        return loss

    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")

    def _shared_eval(self, batch, batch_idx, prefix):
        batch_rfg, batch_targets = batch
        loss = self.loss_funciton(batch_rfg, batch_targets)
        self.log(f"{prefix}_loss", loss)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

## Config

In [ ]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

In [ ]:
############
recfldgrn_list = ['P@PatEcInfo-InfoGrn',  'P-EC-PNSect@AllText-TknzGrn']
############

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'ReMax',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################


############################
default_E_subunitName = 'EL'
default_R_subunit_name = 'RL'
default_MR_subunit_name = 'ML'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 
############################

In [ ]:

full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]
df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunitName)
df_SubUnit['SubUnit_BasicNN_List'] = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), axis = 1)
df_SubUnit['SubUnit_DefaultBasicNN_List'] = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)
df_SubUnit['SubUnit_BasicNN_Config_List'] = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, ), axis = 1)
df_Embed_SubUnit = df_SubUnit
df_Embed_SubUnit

In [ ]:
full_recfldgrn_list_new = [i.replace('Grn', '') for i in full_recfldgrn_list]
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list_new)
df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow = df_dataflow_new.copy()
df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit['SubUnit_BasicNN_List'] = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName),  axis = 1)
df_SubUnit['SubUnit_DefaultBasicNN_List'] = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)
df_SubUnit['SubUnit_BasicNN_Config_List'] = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)
df_SubUnit

In [ ]:
df_Embed_SubUnit = df_Embed_SubUnit
df_Repr_SubUnit = df_Repr_SubUnit
output_name = 'B-P'
embed_size = embed_size
output_size = 1
actn_fn_name = 'Sigmoid' # torch.nn.Sigmoid()
loss_fn_name = 'BCELoss' # torch.nn.BCELoss()

## Init

In [ ]:
Model = RecModelPL(df_Embed_SubUnit, df_Repr_SubUnit,  actn_fn_name, 
                   loss_fn_name, output_name, embed_size, output_size)

## Input

In [ ]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

print(Tensor_folder)
print(recfldgrn_list)
print(full_recfldgrn_list)

In [ ]:
# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')

In [ ]:
for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

In [ ]:
# input
[i for i in batch_rfg]

In [ ]:
# # REPR output

# REPR_TENSOR = Model.get_REPR_TENSOR(batch_rfg)

# [i for i in REPR_TENSOR]

In [ ]:
# Model.output_name

In [ ]:
# [i for i in batch_rfg]

In [54]:
# probs = Model(batch_rfg)
# probs

In [55]:
# Model

# Test PyTorch-Lightning

In [56]:
num_epochs = 2
num_gpus = 0
checkpoint_path = 'checkpoint/'

In [57]:
len(dataloader)

2

In [58]:
batch_rfg, batch_y = batch

In [59]:
batch_y.shape

torch.Size([2, 1])

In [60]:
trainer = pl.Trainer(max_epochs = num_epochs, default_root_dir = checkpoint_path)
trainer.fit(model = Model, train_dataloaders = dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/floydluo/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name        | Type            | Params
------------------------------------------------
0 | EmbedBlock  | EmbedBlockLayer | 109 M 
1 | ReprBlock   | ReprBlockLayer  | 1.5 K 
2 | OutputBlock | Linear          | 129   
3 | actn_method | Sigmoid         | 0     
4 | loss_method | BCELoss         | 0     
------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.364   Total estimated model params size (MB)
/Users/floydluo/opt/miniconda3/envs/boost/lib/python3.8/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWar

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [61]:
# test the model
# trainer.test(model, dataloaders=DataLoader(test_set))

trainer.test(model = Model, dataloaders = dataloader)

/Users/floydluo/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/Users/floydluo/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.8326095342636108
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.8326095342636108}]